# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="034_rerun_400_restaurants",
    RUN_DESCRIPTION="""
# Objective
Re-run with 400 restaurants (instead of 100 misc. biz previously)

# Implementation

## Changelog
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=False,
    RECREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    DEBUG=False,
)

ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "034_rerun_400_restaurants",
  "RUN_DESCRIPTION": "\n# Objective\nRe-run with 400 restaurants (instead of 100 misc. biz previously)\n\n# Implementation\n\n## Changelog\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_RETRIEVAL_EVAL_DATASET": false,
  "RECREATE_RESPONSE_EVAL_DATASET": false,
  "RECREATE_INDEX": false
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-09-08 09:09:42.043 | INFO     | src.run.cfg:init:151 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix
E0000 00:00:1725761386.883431 4026810 chttp2_server.cc:1143] UNKNOWN:No address added out of total 1 resolved for '[::]:4317' {created_time:"2024-09-08T09:09:46.883429787+07:00", children:[UNKNOWN:Failed to add any wildcard listeners {created_time:"2024-09-08T09:09:46.883423025+07:00", children:[UNKNOWN:Unable to configure socket {created_time:"2024-09-08T09:09:46.883391645+07:00", fd:65, children:[UNKNOWN:bind: Address already in use (98) {created_time:"2024-09-08T09:09:46.883370776+07:00"}]}, UNKNOWN:Unable to configure socket {fd:65, created_time:"2024-09-08T09:09:46.883421572+07:00", children:[UNKNOWN:bind: Address already in use (98) {created_time:"2024-09-08T09:09:46.883418636+07:00"}]}]}]}
ERROR:    Traceback (most recent call last):
  File "/home/dvquys/frostmourne/study/vietai-genai03/final-pro

In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "034_rerun_400_restaurants",
    "RUN_DESCRIPTION": "\n# Objective\nRe-run with 400 restaurants (instead of 100 misc. biz previously)\n\n# Implementation\n\n## Changelog\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_RETRIEVAL_EVAL_DATASET": false,
    "RECREATE_RESPONSE_EVAL_DATASET": false,
    "RECREATE_INDEX": false
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/034_rerun_400_restaurants/storage_context",
  "vector_db": "qdrant",
  "db_collection": "review_rec_bot__034_rerun_400_restaurants",
  "db_collection_fp": "data/034_rerun_400_restaurants/chroma_db",
  "notebook_cache_dp": "data/034_rerun_400_restaurants",
  "data_fp": "../data/yelp_dataset/sample/sample_400_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "openai",
    "llm_model_name": "gpt-4o-mini",
    "embedding_provider":

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    # ChromaDB allows only str, int, float dtypes to be indexed as metadata
    # biz_categories=lambda df: df['biz_categories'].str.split(', '),
    biz_categories=lambda df: df['biz_categories'].astype(str),
    biz_attributes=lambda df: df['biz_attributes'].astype(str),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S'),
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-09-08 09:09:47.954 | INFO     | __main__:<module>:9 - [COLLECT] len(data)=37281


review_id                                      40thYphUgIfvJq17QCfTwA
user_id                                        QzCEzH3R7Z6erOGLr3t55Q
business_id                                    0pMj5xUAecW9o1P35B0AMw
review_stars                                                        5
useful                                                              1
funny                                                               0
cool                                                                1
text                Great staff always helps and always nice. Alwa...
date                                              2017-05-26T13:10:24
biz_name                                                         Wawa
biz_address                                        2544 W Main Street
biz_city                                                   Norristown
biz_state                                                          PA
biz_postal_code                                                 19403
biz_latitude        

In [12]:
data.dtypes

review_id            object
user_id              object
business_id          object
review_stars          int64
useful                int64
funny                 int64
cool                  int64
text                 object
date                 object
biz_name             object
biz_address          object
biz_city             object
biz_state            object
biz_postal_code      object
biz_latitude        float64
biz_longitude       float64
biz_stars           float64
biz_review_count      int64
biz_is_open           int64
biz_attributes       object
biz_categories       object
biz_hours            object
dtype: object

In [13]:
input_data = data
if ARGS.TESTING:
    input_data = data[:30]  # 30 examples to avoid bm25 retreiver error...
logger.info(f"[COLLECT] {len(input_data)=}")

2024-09-08 09:09:52.714 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=37281


In [14]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

In [17]:
input_data.loc[lambda df: df['biz_name'].eq('Red Hook Coffee & Tea')]

,review_id,user_id,business_id,review_stars,useful,funny,cool,text,date,biz_name,...,biz_state,biz_postal_code,biz_latitude,biz_longitude,biz_stars,biz_review_count,biz_is_open,biz_attributes,biz_categories,biz_hours
53,yyAycjhj9pBUWYlZd75IZg,rZ4rHwgqTnP4NUcpG_MQvw,bTve2mwLk5Zc01vRKqc2KQ,5,0,0,0,This might be my favorite coffee shop in the c...,2016-05-07T01:15:03,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
219,imLgc2BetqIr8jfvOoIQsQ,4Ln38VKTbX8wxfOvIwHiAw,bTve2mwLk5Zc01vRKqc2KQ,4,2,0,0,New to the area and was looking for a good bru...,2014-08-24T23:55:17,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
311,sKVCrT5KwXIZ7qaepLecIQ,B4Ijd1rmnzAfrP3WVrdo8g,bTve2mwLk5Zc01vRKqc2KQ,5,0,0,0,Red Hook is my jam! This place focuses on the...,2016-01-04T07:12:18,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
377,sePofxLff292WFC5vq2I-A,jjOp1erNIM5xgt_eIPwhDg,bTve2mwLk5Zc01vRKqc2KQ,4,0,0,0,"Had a maple, cayenne latte- spicy and deliciou...",2014-12-17T20:45:28,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
563,NEw0f7z9F8WHE5uU3dWq3w,zY8LmTD4hS7GS_QETgjmAQ,bTve2mwLk5Zc01vRKqc2KQ,5,0,0,0,I love this place!! They have really creative ...,2017-05-13T19:12:20,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36110,xvnP05xKgSBn9U2EAHbbAg,h3adLEYDv7Uw1PP6caVQuQ,bTve2mwLk5Zc01vRKqc2KQ,5,0,0,0,Cozy little spot. Good coffee and food. Hips...,2016-11-26T00:26:44,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
36302,_WrxVE-Q1_aOkJZEKDO3Lw,pZg9_Qbqb7hM2eDinhcVfA,bTve2mwLk5Zc01vRKqc2KQ,1,0,0,0,I got the egg and cheese sandwich. I never lea...,2021-07-08T16:28:49,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
36505,19EgoYFlxrmuVyQ_Jr10dQ,urKv7EWGUeKvW-hxGy8yDQ,bTve2mwLk5Zc01vRKqc2KQ,3,0,0,0,The coffee here is good and everything I've or...,2012-11-11T20:24:45,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."
36583,d3x6mTcPp1pk1-6m1GPaVQ,VP0oebWjhOr6RcYaetpk-w,bTve2mwLk5Zc01vRKqc2KQ,5,2,0,1,HOly Crap! How have I not known about this pl...,2014-01-12T20:10:02,Red Hook Coffee & Tea,...,PA,19147,39.938552,-75.149636,4.5,183,1,"{'Alcohol': ""'none'"", 'Ambience': ""{'romantic'...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Friday': '7:0-18:0', 'Monday': '7:0-18:0', '..."


# Prepare documents

In [15]:
from llama_index.core import Document

documents = []
metadata_excluded_cols = ('biz_hours', 'text')
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories", "business_id"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in metadata_excluded_cols}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/37281 [00:00<?, ?it/s]

2024-09-08 08:32:44.342 | INFO     | __main__:<module>:21 - [COLLECT] len(documents)=37281


#### Check document embedding text

In [16]:
from llama_index.core.schema import MetadataMode

In [17]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

business_id: 0pMj5xUAecW9o1P35B0AMw
review_stars: 5
biz_name: Wawa
biz_address: 2544 W Main Street
biz_city: Norristown
biz_state: PA
biz_categories: Food, Coffee & Tea, Gas Stations, Restaurants, Automotive, Sandwiches, Convenience Stores

Great staff always helps and always nice. Always clean and huge store size. Stays stocked with everything you need. The staff always helps make my custom breakfast bagel


# Set up LLM

In [18]:
llm, embed_model = cfg.setup_llm()

WARNI [sentence_transformers.SentenceTransformer] You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [19]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "openai",
  "llm_model_name": "gpt-4o-mini",
  "embedding_provider": "huggingface",
  "embedding_model_name": "./data/finetune_embedding/finetuned_model",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [20]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [21]:
if cfg.vector_db == 'chromadb':
    from src.run.vector_db import ChromaVectorDB as VectorDB
elif cfg.vector_db == 'qdrant':
    from src.run.vector_db import QdrantVectorDB as VectorDB

In [22]:
vector_db = VectorDB(cfg)
vector_store = vector_db.vector_store
db_collection_count = vector_db.doc_count
logger.info(f"{db_collection_count=}")

2024-09-08 08:32:48.093 | INFO     | src.run.vector_db.qdrant:_setup_db:58 - Using existing Qdrant collection: review_rec_bot__034_rerun_400_restaurants
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.
2024-09-08 08:32:48.099 | INFO     | __main__:<module>:4 - db_collection_count=72717


# Index Embeddings

In [23]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
# from llama_index.core.node_parser import SentenceSplitter

In [24]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [25]:
t0 = time.perf_counter()

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=False)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-09-08 08:32:48.217 | INFO     | __main__:<module>:4 - Loading Storage Context from data/034_rerun_400_restaurants/storage_context...


In [26]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-09-08 08:33:41.218 | INFO     | __main__:<module>:1 - Indexing 37281 documents into VectorStoreIndex took 53s
2024-09-08 08:33:41.218 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=72717


# Analyze Chunks

In [27]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': '40thYphUgIfvJq17QCfTwA', 'user_id': 'QzCEzH3R7Z6erOGLr3t55Q', 'business_id': '0pMj5xUAecW9o1P35B0AMw', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2017-05-26T13:10:24', 'biz_name': 'Wawa', 'biz_address': '2544 W Main Street', 'biz_city': 'Norristown', 'biz_state': 'PA', 'biz_postal_code': '19403', 'biz_latitude': 40.141292, 'biz_longitude': -75.389286, 'biz_stars': 3.5, 'biz_review_count': 8, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\':

# Construct Retriever

In [28]:
# from src.run.retriever.auto_retrieval import get_retriever
from src.run.retriever.retriever import get_retriever

In [29]:
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = get_retriever(cfg, index)

In [30]:
query = "Nashville, food"
vector_retrieval_results = vector_retriever.retrieve(query)
for result in vector_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

**Node ID:** f9929e35-9ef3-4a85-aeca-590430962e61<br>**Similarity:** 0.67196095<br>**Text:** The food was decent once we got it but honestly, not all that great considering the hassle you have to go through to get it and eat. There are a million great places to eat in Nashville...I'd put this low on the list.<br>**Metadata:** {'review_id': 'zf2I-PCmKYTh2pWZgJ2PZw', 'user_id': 'K1tkMX_b5NAirPCyfyZvyA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 2, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2019-08-19T19:05:21', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners'}<br>

**Node ID:** 946c0bf2-d489-40d4-9669-df97afd98461<br>**Similarity:** 0.6543033<br>**Text:** Best place to eat in Nashville! Amazing, flavorful food.<br>**Metadata:** {'review_id': 'vjJbkZJzZWa693nI2ZQzPw', 'user_id': 'dUQHBrjgye9GbXIQEsDV1g', 'business_id': 'I2CxrvM0BGH1svfxqGGD2g', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-06-13T00:55:10', 'biz_name': 'M.L.Rose Craft Beer & Burgers - Melrose', 'biz_address': '2535 8th Ave S, Ste 107', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37204', 'biz_latitude': 36.1226231735, 'biz_longitude': -86.7761365195, 'biz_stars': 3.5, 'biz_review_count': 472, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': \'False\', \'BYOBCorkage\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': True, \'friday\': True, \'wednesday\': False, \'thursday\': True, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': \'False\', \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': True, \'dinner\': True, \'brunch\': None, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': None, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'free\'"}', 'biz_categories': 'Automotive, Restaurants, Sandwiches, Bars, Nightlife, Pubs, American (Traditional), Burgers, Beer Bar, Sports Bars, Breakfast & Brunch'}<br>

**Node ID:** dc81c000-bfea-4530-8048-9de778762dde<br>**Similarity:** 0.6495869<br>**Text:** First was on a farm in Connecticut where I'm pretty sure the milk was straight from the cows I could smell while eating, so I'm holding pretty high standards. This is a must go to for when your in Nashville!<br>**Metadata:** {'review_id': 'WTxz8Sz6wsUfSK0PSesm-A', 'user_id': 'NQzJrHygqgqc1Xa_08JOoQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2014-10-18T23:07:29', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** abd226f0-8e2b-44f5-bf74-a105e6ddae05<br>**Similarity:** 0.6396048<br>**Text:** A must stop in Nashville<br>**Metadata:** {'review_id': 'BbJRG7HkGx0S3SuvvpVvIQ', 'user_id': '26nVu2FZTrYt3PEMXLCR0A', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2013-09-28T04:48:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 93cb52b1-7093-4672-ba1b-fc0c87594501<br>**Similarity:** 0.6393957<br>**Text:** This is one of my favorite ice cream places in Nashville.  They have really good, quality food.<br>**Metadata:** {'review_id': '6oiQ2B371hyMFGH5do6GZQ', 'user_id': 'aX02PYvbsMXwCzndTuys6w', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-07-16T18:07:47', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

...

**Node ID:** 9dbe0f50-c715-4898-b524-ec098fb1cb63<br>**Similarity:** 0.59211415<br>**Text:** Found this place on instagram. Definitely worth checking out. The food here is really good!  The portions are generous so a plate will fill you up.  I had the Jerk chicken with rice and peas and my girlfriend had the oxtails with cabbage and mac n cheese. Was one of the best Jamaican meals I've had in Nashville.  I would definitely recommend this spot.<br>**Metadata:** {'review_id': 'fB9oGKCo2laqVc_1bdw8mA', 'user_id': '7ZyAeMSbq32GkLnFyhCT4Q', 'business_id': '1dSKEitDDgIkaApe6UNMSA', 'review_stars': 5, 'useful': 8, 'funny': 3, 'cool': 5, 'date': '2020-10-11T21:58:09', 'biz_name': 'The Pepper Pott', 'biz_address': '4611 Alabama Ave', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37209', 'biz_latitude': 36.1530043, 'biz_longitude': -86.8433279, 'biz_stars': 5.0, 'biz_review_count': 18, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'none\'", \'Ambience\': None, \'BYOB\': \'False\', \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': \'False\', \'DogsAllowed\': \'True\', \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': \'False\', \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': None, \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': None, \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'no\'"}', 'biz_categories': 'Caribbean, Food, Restaurants, Food Trucks'}<br>

**Node ID:** ff704f2c-4778-4ba1-a6a5-e9c47bb6079e<br>**Similarity:** 0.5920291<br>**Text:** Prices are cheap as well! This was a great sweets option in Downtown Nashville.<br>**Metadata:** {'review_id': 'UB_KQiHpvPx8We7bpHb92w', 'user_id': 'hYWwocw-J4KdyGLrUlOYXA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 1, 'funny': 1, 'cool': 1, 'date': '2016-10-16T21:29:55', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 7fec5b5c-fae7-4ae5-82a9-d8b7072f5ea6<br>**Similarity:** 0.59196264<br>**Text:** The froyo (frozen yogurt) place charged .70 cents an ounce. 

I'd highly recommend making a drive to the many great Nashville eateries. I've provided a ranking of one's that I visited based on Yelp research and where I was at. 

Rank of dining establishments in Nashville area I visited:

1 being my favorite 

1-Catbird Seat  $$$$
2-Hattie B's  $$
3- Peg Leg Porker $$
4- Fin and Pearl  $$
5-Martin's BBQ   $$
6- Mission BBQ  $$
7-Biscuit Love  $$<br>**Metadata:** {'review_id': 'A2Jn0qWB0jcnS2JfxIGMKA', 'user_id': 'j8m6Qn1aCwSYEBbXlVGgGA', 'business_id': 'ORL4JE6tz3rJxVqkdKfegA', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-06-26T17:51:32', 'biz_name': 'Gaylord Opryland Resort & Convention Center', 'biz_address': '2800 Opryland Dr', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37214', 'biz_latitude': 36.2115923083, 'biz_longitude': -86.6943191727, 'biz_stars': 3.0, 'biz_review_count': 1639, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "\'full_bar\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': True, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': False}", \'BYOB\': \'True\', \'BYOBCorkage\': "\'no\'", \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': True, \'wednesday\': False, \'thursday\': False, \'sunday\': True, \'saturday\': True}", \'BikeParking\': None, \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': \'True\', \'Caters\': \'True\', \'CoatCheck\': \'True\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': True, \'background_music\': True, \'no_music\': False, \'jukebox\': False, \'live\': True, \'video\': False, \'karaoke\': True}", \'NoiseLevel\': "u\'very_loud\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'3\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'outdoor\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'paid\'"}', 'biz_categories': 'Venues & Event Spaces, Performing Arts, Arts & Entertainment, Hotels & Travel, Food, Convenience Stores, American (New), Beauty & Spas, Restaurants, Museums, Event Planning & Services, Hotels, Cinema, Resorts, Day Spas'}<br>

**Node ID:** 937a334b-8c67-48c8-80c5-feec478d9ac6<br>**Similarity:** 0.5913899<br>**Text:** Awesome place...Ice Cream is vey very delicious and Spanish chocolate is authentic and sooo good. Highly recommended one of the best in Nashville.<br>**Metadata:** {'review_id': 'T6shYaRHk6U5db9yA6PR5g', 'user_id': 'mQfj1Q0fvSes0ZJA3WP1OA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-01-26T02:44:32', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** d8ff5210-9232-4140-aea8-a672e4733de5<br>**Similarity:** 0.5898651<br>**Text:** The yellow cake batter was heaven sent as well.  Went here everyday while in Nashville.  Yum!<br>**Metadata:** {'review_id': '2Mb--4ZqhZQaQpz2fohRTw', 'user_id': 'TEHiUF-e8pa37rByg15MmA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 1, 'date': '2011-05-16T01:29:04', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

In [31]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, food")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** 39c9abe1-8d5f-4ecd-9e93-b4a5bd82df11<br>**Similarity:** 3.8010964393615723<br>**Text:** When I visit Nashville and have a taste for Indian food this is the place to go. Good food, nice people and the best Biryani in Nashville.<br>**Metadata:** {'review_id': 'MYYAcJ2y9-TkWA81mCx2OA', 'user_id': 'cNXDoeoV848iZI2ca8T_gQ', 'business_id': 'SwWfW3vBn5QkDE7T3urGAg', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-08-30T23:49:52', 'biz_name': 'Persis Biryani Indian Grill', 'biz_address': '630 S Mount Juliet Rd, Ste 310', 'biz_city': 'Mount Juliet', 'biz_state': 'TN', 'biz_postal_code': '37122', 'biz_latitude': 36.162142, 'biz_longitude': -86.516006, 'biz_stars': 4.5, 'biz_review_count': 246, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "\'beer_and_wine\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': False, u\'casual\': True, u\'touristy\': False, u\'trendy\': None, u\'intimate\': None, u\'romantic\': False, u\'classy\': None, u\'upscale\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': None, \'validated\': False, \'lot\': None, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': False, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'False\', \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'free\'"}', 'biz_categories': 'Indian, Restaurants'}<br>

**Node ID:** 37ca6f43-e361-4ab3-9818-9f965e956ee6<br>**Similarity:** 3.600358486175537<br>**Text:** Excellent find in Nashville. Lorraine is an excellent server. I WILL be back to this place the next time I am in Nashville (in about three weeks). Best Happy Hour in Nashville if you want to relax. Great food (Gyro).<br>**Metadata:** {'review_id': '2bR7q3vVxFJ4IkLqvZTB8A', 'user_id': 'qcH4XOYo8AE9pkvAsYPOsg', 'business_id': 'Gki1ureeRKqlIkQ-AELDVg', 'review_stars': 5, 'useful': 1, 'funny': 1, 'cool': 1, 'date': '2016-01-15T00:25:04', 'biz_name': 'Brugada', 'biz_address': '204 Commerce St', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.163323, 'biz_longitude': -86.7765917, 'biz_stars': 3.5, 'biz_review_count': 49, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': None, \'hipster\': None, \'romantic\': False, \'divey\': None, \'intimate\': None, \'trendy\': None, \'upscale\': False, \'classy\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': True, \'wednesday\': False, \'thursday\': True, \'sunday\': False, \'saturday\': True}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'True\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': False, \'dinner\': False, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': True, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': True, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'yes\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Restaurants, Nightlife, Dance Clubs, Falafel, Lounges, Hookah Bars, Bars, Mediterranean'}<br>

**Node ID:** 946c0bf2-d489-40d4-9669-df97afd98461<br>**Similarity:** 3.4344911575317383<br>**Text:** Best place to eat in Nashville! Amazing, flavorful food.<br>**Metadata:** {'review_id': 'vjJbkZJzZWa693nI2ZQzPw', 'user_id': 'dUQHBrjgye9GbXIQEsDV1g', 'business_id': 'I2CxrvM0BGH1svfxqGGD2g', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-06-13T00:55:10', 'biz_name': 'M.L.Rose Craft Beer & Burgers - Melrose', 'biz_address': '2535 8th Ave S, Ste 107', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37204', 'biz_latitude': 36.1226231735, 'biz_longitude': -86.7761365195, 'biz_stars': 3.5, 'biz_review_count': 472, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': \'False\', \'BYOBCorkage\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': True, \'friday\': True, \'wednesday\': False, \'thursday\': True, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': \'False\', \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': True, \'dinner\': True, \'brunch\': None, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': None, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'free\'"}', 'biz_categories': 'Automotive, Restaurants, Sandwiches, Bars, Nightlife, Pubs, American (Traditional), Burgers, Beer Bar, Sports Bars, Breakfast & Brunch'}<br>

**Node ID:** f07f20da-e522-4266-81c6-96c7cc8a8651<br>**Similarity:** 3.378657341003418<br>**Text:** Restaurant food is NOT worth the cost. Stay closer to Nashville, better food and more convenient.<br>**Metadata:** {'review_id': 'IdOl4bUreSozM8DoLTDXXQ', 'user_id': 'vEYec9YhfU7KK9NQAGGpIw', 'business_id': 'ORL4JE6tz3rJxVqkdKfegA', 'review_stars': 1, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-11-15T16:28:39', 'biz_name': 'Gaylord Opryland Resort & Convention Center', 'biz_address': '2800 Opryland Dr', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37214', 'biz_latitude': 36.2115923083, 'biz_longitude': -86.6943191727, 'biz_stars': 3.0, 'biz_review_count': 1639, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "\'full_bar\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': True, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': False}", \'BYOB\': \'True\', \'BYOBCorkage\': "\'no\'", \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': True, \'wednesday\': False, \'thursday\': False, \'sunday\': True, \'saturday\': True}", \'BikeParking\': None, \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': \'True\', \'Caters\': \'True\', \'CoatCheck\': \'True\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': True, \'background_music\': True, \'no_music\': False, \'jukebox\': False, \'live\': True, \'video\': False, \'karaoke\': True}", \'NoiseLevel\': "u\'very_loud\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'3\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'outdoor\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'paid\'"}', 'biz_categories': 'Venues & Event Spaces, Performing Arts, Arts & Entertainment, Hotels & Travel, Food, Convenience Stores, American (New), Beauty & Spas, Restaurants, Museums, Event Planning & Services, Hotels, Cinema, Resorts, Day Spas'}<br>

...

**Node ID:** 36bd2177-46ee-4bee-a77f-88309180cf69<br>**Similarity:** 2.804476737976074<br>**Text:** Best new restaurant in Nashville, hands down. The food we had was solid but we just went with a bunch of small plates that we shared.<br>**Metadata:** {'review_id': 'FmTj5ll28M5HudeZh6L6Fg', 'user_id': 'zQPhXL3k70xrL9lmgACHFA', 'business_id': 'tMkwHmWFUEXrC9ZduonpTg', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-11-18T19:18:30', 'biz_name': 'The Green Pheasant', 'biz_address': '215 1st Ave S', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1598858, 'biz_longitude': -86.7731974, 'biz_stars': 4.0, 'biz_review_count': 161, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': True, \'lot\': False, \'valet\': True}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': None, \'dinner\': True, \'brunch\': None, \'breakfast\': None}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': None, \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'None\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Japanese, Seafood'}<br>

**Node ID:** 7ca180a2-eb7a-42ad-ad8c-01d475ea1f39<br>**Similarity:** 2.7998337745666504<br>**Text:** It's not worth the time or the money. I hope that the Burger Up in Nashville is still going strong. If not, I would be surprised to find it here when we visit Nashville again.<br>**Metadata:** {'review_id': 'v27K_tSZaSCxRkpxsx9y7A', 'user_id': 'KyQG7pkQRHg3mYxCIEW6lw', 'business_id': 'skN2XhKXlcjf53uIwzAedw', 'review_stars': 2, 'useful': 2, 'funny': 0, 'cool': 0, 'date': '2013-04-01T00:54:00', 'biz_name': 'Burger Up - Franklin', 'biz_address': '401 Cool Springs Blvd', 'biz_city': 'Franklin', 'biz_state': 'TN', 'biz_postal_code': '37067', 'biz_latitude': 35.9459704793, 'biz_longitude': -86.8279486659, 'biz_stars': 3.5, 'biz_review_count': 329, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': True, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'True\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Burgers, Desserts, Restaurants, Food, Salad'}<br>

**Node ID:** 12f2a7b3-7be1-4bce-8619-7593001b7023<br>**Similarity:** 2.772703170776367<br>**Text:** Best Tacos I have ever had!! We were looking for Mexican food in Nashville and just happened upon this place, so glad we did! Drinks were great and food was fabulous!<br>**Metadata:** {'review_id': '-ubqHrNTor696KDz1tJuBw', 'user_id': 'JPJJriYquD_6BWTBk1LxWw', 'business_id': 'X_E7U2lVNEwbH_gpOBmAHQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-02-03T01:20:55', 'biz_name': "Cerveza Jack's", 'biz_address': '135 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.162799, 'biz_longitude': -86.776016, 'biz_stars': 4.0, 'biz_review_count': 182, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{u\'divey\': None, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': True, u\'intimate\': False, u\'romantic\': False, u\'classy\': None, u\'upscale\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': True, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': True}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': None, \'street\': True, \'validated\': None, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': \'True\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': True, \'lunch\': True, \'dinner\': True, \'brunch\': None, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': True, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'loud\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Nightlife, Restaurants, Bars, Mexican'}<br>

**Node ID:** 827f6bdd-eec4-4318-af02-cbb72e7a95dd<br>**Similarity:** 2.760499954223633<br>**Text:** I visited them again the next day and the food was consistent. Overall good tasty south Indian food, friendly owner, piping hot food which makes Dosa Hut a must visit in Nashville if you love authentic Indian food.<br>**Metadata:** {'review_id': 'n9uqOE--3uOK0OnmHrz70A', 'user_id': 'hiWFw6N6-co5R0x3SQjuSQ', 'business_id': '6mCsrUp9Vl4iXMbOxWtVjA', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-11-28T13:29:08', 'biz_name': 'Dosa Hut Indian Grill', 'biz_address': '4047 Nolensville Pike', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37211', 'biz_latitude': 36.0810108, 'biz_longitude': -86.7280648, 'biz_stars': 4.0, 'biz_review_count': 99, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "\'none\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'1\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Indian'}<br>

**Node ID:** ec6eafcf-0d14-4c28-b4fa-2b0c210d343f<br>**Similarity:** 2.7584757804870605<br>**Text:** Great vibe, friendly servers/staff, good food. If you're reading this, don't come here, and while you're at it don't move to Nashville.<br>**Metadata:** {'review_id': 'AePqOTZ8_3bPGhapO_athA', 'user_id': 'V2-3_38KlNlRK0oQjUNnnQ', 'business_id': 'dD2p903p8lU0IgXT3OFluA', 'review_stars': 1, 'useful': 0, 'funny': 3, 'cool': 0, 'date': '2015-10-13T17:24:27', 'biz_name': 'Edgehill Cafe', 'biz_address': '1201 Villa Pl', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37212', 'biz_latitude': 36.1429685, 'biz_longitude': -86.7923549, 'biz_stars': 3.5, 'biz_review_count': 245, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': False, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'1\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Breakfast & Brunch, Restaurants, Food, Cafes, Coffee & Tea, American (Traditional), Sandwiches'}<br>

In [32]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [33]:
query = "What are some highly reviewed places to enjoy cold brew coffee in Santa Barbara?"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.LLM))
    print("\n")

Generated queries:
cold brew coffee Santa Barbara
"I visited a fantastic café in Santa Barbara that had the best cold brew coffee I've ever tasted! The atmosphere was cozy, and the staff was super friendly. Highly recommend checking it out!"


**Node ID:** 60dc1450-1ab0-4440-9c89-3cff69b8d381<br>**Similarity:** 0.06314426076226826<br>**Text:** A must stop in Santa Barbara. Loved their coffee!<br>

business_id: -kY_HDP7IMvGl-kBIZVU4A
review_stars: 5
biz_name: Dune Coffee Roasters - Anacapa
biz_address: 528 Anacapa St
biz_city: Santa Barbara
biz_state: CA
biz_categories: Coffee & Tea, Coffee Roasteries, Food

A must stop in Santa Barbara. Loved their coffee!




**Node ID:** d433f725-1fa2-4d49-9d2b-e4b8af05d35b<br>**Similarity:** 0.059629259629259634<br>**Text:** Delicious local coffee! A must stop when I'm in Santa Barbara!<br>

business_id: -kY_HDP7IMvGl-kBIZVU4A
review_stars: 5
biz_name: Dune Coffee Roasters - Anacapa
biz_address: 528 Anacapa St
biz_city: Santa Barbara
biz_state: CA
biz_categories: Coffee & Tea, Coffee Roasteries, Food

Delicious local coffee! A must stop when I'm in Santa Barbara!




...

business_id: -kY_HDP7IMvGl-kBIZVU4A
review_stars: 5
biz_name: Dune Coffee Roasters - Anacapa
biz_address: 528 Anacapa St
biz_city: Santa Barbara
biz_state: CA
biz_categories: Coffee & Tea, Coffee Roasteries, Food

We visited this coffee shop four times over the week that we were in Santa Barbara. Fabulous espresso drinks. It was the best non-dairy milk cappuccino I've ever had (and I've been on this mission for awhile). Try the oatmilk!




**Node ID:** cce23e7b-8e32-44d2-9dd1-b8c2a781c718<br>**Similarity:** 0.02024827024827025<br>**Text:** The coffee is just as great as the original State Street location - easily the best in Santa Barbara - and there's plenty of parking, too. The decor is very clean and contemporary and the ambiance is just as friendly as always.<br>

business_id: -kY_HDP7IMvGl-kBIZVU4A
review_stars: 5
biz_name: Dune Coffee Roasters - Anacapa
biz_address: 528 Anacapa St
biz_city: Santa Barbara
biz_state: CA
biz_categories: Coffee & Tea, Coffee Roasteries, Food

The coffee is just as great as the original State Street location - easily the best in Santa Barbara - and there's plenty of parking, too. The decor is very clean and contemporary and the ambiance is just as friendly as always.




**Node ID:** bd649ea4-2144-4011-b9c1-bd2784972ab5<br>**Similarity:** 0.01980974796508777<br>**Text:** A place to go when I'm too lazy to bring out my french press and make coffee for myself.  Since I no longer live in close proximity to Santa Barbara, I will miss this place.  So for those who live nearby, you're very lucky.<br>

business_id: -kY_HDP7IMvGl-kBIZVU4A
review_stars: 5
biz_name: Dune Coffee Roasters - Anacapa
biz_address: 528 Anacapa St
biz_city: Santa Barbara
biz_state: CA
biz_categories: Coffee & Tea, Coffee Roasteries, Food

A place to go when I'm too lazy to bring out my french press and make coffee for myself.  Since I no longer live in close proximity to Santa Barbara, I will miss this place.  So for those who live nearby, you're very lucky.




## Retrieval Evaluation

### Synthetic

In [34]:
from src.run.eval import RetrievalEvaluator

In [35]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-09-08 08:34:21.248 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:79 - Loading retrieval_eval_nodes from data/034_rerun_400_restaurants/retrieval_synthetic_eval_dataset.json...
2024-09-08 08:34:21.249 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:85 - Loading existing synthetic retrieval eval dataset at data/034_rerun_400_restaurants/retrieval_synthetic_eval_dataset.json...


In [36]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'db577289-00d1-4fdd-ae1b-1d3da9ab0c33': 'What are some restaurants known for their flavorful Italian dishes?',
 '143e1bf3-1daf-41d8-afdb-356ed193ad5e': 'What are some restaurants known for their fresh seafood and flavorful dishes?',
 'b6cf5590-b0ea-4afc-9cba-a57038a24d77': 'What are some restaurants known for their bright and clean decor?',
 '63534f01-b0fb-426a-8e73-79ab0d8646f1': 'What are some hotels known for their reliable lost and found services?',
 'd1834941-0dbb-4fe2-8af8-5b7c7d949b87': 'What are some restaurants known for excellent customer service?',
 '6c7d990e-c05d-4d19-99f2-3107394c1337': 'What are some hotels that provide better accessibility features and reliable maintenance services?',
 '1346cbb4-f743-4924-b25b-dd020a360e56': 'What are some popular brunch spots that offer unique dishes like fig French toast?',
 'bf9150d0-bb9e-469b-89e5-200c20465f9b': 'What are some restaurants known for exceptional customer service?',
 'e6960def-c4d2-4f90-b9e9-c6b380785d3d': 'What are so

In [37]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
Italian restaurants flavorful dishes
Generated queries:
fresh seafood restaurants flavorful dishes
"I recently dined at Ocean's Delight, and their seafood was incredibly fresh! The grilled salmon was bursting with flavor, and the shrimp scampi was a standout. Highly recommend for anyone who loves seafood!"
Generated queries:
bright clean decor restaurants
"I recently visited a restaurant with stunning bright decor and a clean atmosphere. The ambiance was refreshing, and it made the dining experience even more enjoyable!"
Generated queries:
reliable lost and found hotels
"I stayed at the Grand Plaza Hotel, and I was impressed by their lost and found services. I accidentally left my jacket behind, and they contacted me right away to return it. Highly recommend!"
Generated queries:
excellent customer service restaurants
Generated queries:
accessible hotels maintenance services
Generated queries:
popular brunch spots unique dishes fig French toast
"I recently visited a b

In [38]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_50_retrieval_eval,0.6,0.308822,0.012,0.6,0.308822,0.028994


In [39]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some restaurants known for their flav...,[8a33d9bb-b763-4f92-a483-c0e106e4c9ca],"[4461fe9e-c578-47ac-bf3d-3cfe3d6db2fb, 8a33d9b...",1.0,0.500000,0.02,1.0,0.500000,0.048918
1,What are some restaurants known for their fres...,[d41df8b7-2dd8-4571-8432-f90cc84bbde6],"[6d585059-db4c-4ac6-807e-b209682359bc, d10ecda...",0.0,0.000000,0.00,0.0,0.000000,0.000000
2,What are some restaurants known for their brig...,[2483c61f-6545-4ad5-8c58-08e485e26f46],"[240832b9-fc5e-47e3-9d96-f2dfbbe9b05b, 25e0333...",1.0,0.333333,0.02,1.0,0.333333,0.038767
3,What are some hotels known for their reliable ...,[5f64b794-1789-4ad1-b9f3-782818e396a5],"[4bfef29a-03a6-42ef-972e-00a49c8e0e16, 0bab15f...",1.0,0.055556,0.02,1.0,0.055556,0.018252
4,What are some restaurants known for excellent ...,[34cf92b6-76dd-48c6-9370-85480e551579],"[b3ba6171-dad4-4a60-8091-13c79e17598f, 03f4a65...",0.0,0.000000,0.00,0.0,0.000000,0.000000
5,What are some hotels that provide better acces...,[7824f18f-50c7-45ef-9431-22ae29402157],"[7824f18f-50c7-45ef-9431-22ae29402157, d2b2a00...",1.0,1.000000,0.02,1.0,1.000000,0.077533
6,What are some popular brunch spots that offer ...,[99a01609-98bc-44d5-8c82-0ccc3a5c1135],"[99a01609-98bc-44d5-8c82-0ccc3a5c1135, 0576cca...",1.0,1.000000,0.02,1.0,1.000000,0.077533
7,What are some restaurants known for exceptiona...,[02a8338b-fcd0-4944-a61f-0b2232065805],"[e1c5125f-34f2-4ac7-85f3-704b04023527, 31ffcd8...",0.0,0.000000,0.00,0.0,0.000000,0.000000
8,What are some highly recommended Italian resta...,[c1f93587-24db-401e-81be-924a94f16552],"[94423b69-eb5c-4db0-8798-00fa6bf3f25d, c1f9358...",1.0,0.500000,0.02,1.0,0.500000,0.048918
9,What are some restaurants that offer local org...,[3e4fdae3-fe9f-4feb-9932-91e1a37e453a],"[8301f24d-9fc7-4a51-9e50-6b520edf340e, deabe87...",0.0,0.000000,0.00,0.0,0.000000,0.000000


#### Error Analysis

In [40]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
1,What are some restaurants known for their fresh seafood and flavorful dishes?,[d41df8b7-2dd8-4571-8432-f90cc84bbde6],"[6d585059-db4c-4ac6-807e-b209682359bc, d10ecdab-05ce-4710-b0d1-43ba71e55808, 194905c2-5fca-4bc9-89f6-827b102bf5fc, 6246d3f7-60f0-477f-bce9-540aabc59788, 80775b68-afbb-4d85-91ed-5c0795ac2920, 621b966a-2b0e-4b5d-a83c-6f9e42694f02, 63c307ad-6be0-4337-812d-df96103e6c26, 8c6fddf2-51af-49b6-8e2f-5980346d4f4b, 2bd1e410-cb6d-4abd-baae-febd139315a1, 1a3f6374-ee6e-4ff8-9d15-023f1b80dac1, 53a39f2e-9ffc-4...",0.0,0.0,0.0,0.0,0.0,0.0
4,What are some restaurants known for excellent customer service?,[34cf92b6-76dd-48c6-9370-85480e551579],"[b3ba6171-dad4-4a60-8091-13c79e17598f, 03f4a65c-8c49-4688-a233-a276d61ad2e4, dc42d8db-3bb3-4716-ac2c-5d11b5fc820c, 09f13a33-2595-4aa0-b4c6-c630d5b67546, 9588c119-f630-49a8-a655-c99667969140, ddc8ddf5-105f-4382-873a-5daf3d4b1d81, a6f39c5a-999d-44b8-ba37-3000f53a9b36, 273b497e-32b0-4633-be66-2b61884e3765, 746d5eba-be7e-4395-bc11-3f77690c095c, e1c5125f-34f2-4ac7-85f3-704b04023527, 72d11f9e-f2ff-4...",0.0,0.0,0.0,0.0,0.0,0.0
7,What are some restaurants known for exceptional customer service?,[02a8338b-fcd0-4944-a61f-0b2232065805],"[e1c5125f-34f2-4ac7-85f3-704b04023527, 31ffcd87-6451-4ae6-89b9-23784c812cb2, 49e48014-a9fc-4289-be70-9a56bd5632e7, c579ddbf-e848-412f-ab1b-290c007d6201, c6440f3a-4c6c-43a8-883b-164bc73bd35e, ddc8ddf5-105f-4382-873a-5daf3d4b1d81, f086d7b3-f558-4cfd-90f5-1dc02d2f49a5, aa102b41-de68-46ca-a508-97588d175988, a6f39c5a-999d-44b8-ba37-3000f53a9b36, c287fb59-9a7d-4643-a7bf-ba3c686b0ba0, 0749c985-404a-4...",0.0,0.0,0.0,0.0,0.0,0.0
9,What are some restaurants that offer local organic foods and have a good outdoor seating area?,[3e4fdae3-fe9f-4feb-9932-91e1a37e453a],"[8301f24d-9fc7-4a51-9e50-6b520edf340e, deabe877-2116-4c4e-bd70-7d485dcaad1a, 2ab497e8-7e35-45a7-8323-970992cf808b, 63b21e4d-ab14-4955-9a44-d5ce5100e4b8, c6c2e125-bfa2-4d86-818f-464cba5af8f2, f630dfd1-9b3b-46c0-a5cb-afa8bfed89f9, d561f61d-5d63-49d9-9952-109bfde66cb3, afef4e5d-5da8-40ae-8293-bdc5a68043f7, 75792b54-aaa3-4b5b-886b-99ad10356c70, 318f45b4-c5e8-4a37-944e-a5de94112fd2, 83c252f5-f21e-4...",0.0,0.0,0.0,0.0,0.0,0.0
11,What are some restaurants known for generous food portions?,[a4de4789-36f1-4651-aace-8604c24e53bd],"[1ff88955-a56d-49e1-86be-c3a6aaa392b1, 1c21fe62-744e-4203-973a-c40a18629e46, ea66daf4-42b5-4516-94f6-f521af9d3414, 8f26f2cf-d9df-4062-a0af-c269197868ac, 644bf239-6815-4660-b2d5-c8ed1d6507fb, c0da3f9e-4702-4f9a-8f47-3404d6739dc3, 0cdfc62f-97d3-44dc-a4c2-c5ecdad83c65, f2b4fbd9-cb22-42b1-b73d-6e40d753bbf5, e51ab50f-4980-4078-8b47-d14bee36ad04, 7591c87c-85c8-4516-8b19-334b6a164118, 8e4036c0-74f1-4...",0.0,0.0,0.0,0.0,0.0,0.0
16,What are some places to avoid visiting in that area?,[a899883b-26ef-4093-9a6e-5fc81e5eeaa3],"[7267492c-f943-4d78-bf55-ccd2a7bba11e, 6eda60ed-24d5-4da2-ac95-af01a45317ce, 40f33a84-83d9-4cf5-a0d9-d6734249136a, 720d7810-5aae-45f7-8be4-88b9da4c6ec7, 459d1676-ca78-4486-9715-bc4b0ea715a4, ce728b0c-10d9-41bb-9efc-cb64292ebd83, 6e5a7910-fb94-4026-a162-ac79faa2a555, ad9c0527-0e1c-4a78-acf8-a8eb9137ac9d, fea47657-38d9-4ea7-b50b-4699d83925ce, 1ae604bf-f751-4ab9-820f-11bb4172c8e6, 8b5aa59c-4869-4...",0.0,0.0,0.0,0.0,0.0,0.0
18,What are some restaurants known for good service and quality food?,[5b6824f2-506e-47a7-8973-fc39704d7a44],"[042c10d7-235e-4619-96c6-0dd5ba2cd8f8, 55314353-43fc-4b80-9bfb-2a4ca1608ed0, 62c847d0-2fbe-4811-9486-2465a53ceaa6, bd5772d6-dd87-444a-bebd-246b1f3a3aee, c76f404f-5825-4b66-9854-a335113498c3, cd3feec0-6b73-4460-8051-f5a4b20ac60d, 971c923f-3e78-43f9-9642-40800780c40b, 3c8becd7-0284-486b-b695-bac2df6908c9, c5208817-1eee-4316-8f93-1ca7550950cf, efd27fd2-9871-4139-a620-28b03a481a87, d8f8185a-ba56-4...",0.0,0.0,0.0,0.0,0.0,0.0
19,What are some restaurants with a great beer selection and a welcoming atmosphere?,[e4cf7b56-8d59-461e-adc9-bc760f3d2b95],"[814ff731-0a29-483e-b74a-a3acd0230c22, 7cca38f3-

In [41]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes[:10]):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some restaurants known for their fresh seafood and flavorful dishes?

- Expected contexts:

business_id: SZU9c8V2GuREDN5KgyHFJw
review_stars: 2
biz_name: Santa Barbara Shellfish Company
biz_address: 230 Stearns Wharf
biz_city: Santa Barbara
biz_state: CA
biz_categories: Live/Raw Food, Restaurants, Seafood, Beer Bar, Beer, Wine & Spirits, Bars, Food, Nightlife

The food definitely did not live up to my expectations! The service was good despite the long wait and huge crowd, but the food itself was subpar. I ate the lobster bisque in a bread bowl and a crab sandwich. The bisque was too tomatoey and I didn't even see and lobster chunks in it, and the bread bowl was NOT fresh. The crab sandwich was decent, but flavorless, you could hardly tell that it was crab. My mother had the lobster roll which was apparently delicious, though ridiculously overpriced at $20. The clam chowder was thick and buttery, and the bread bowl was once again st

In [42]:
def check_retrieval_results(query):
    retrieval_results = retriever.retrieve(query)
    for node in retrieval_results:
        display_source_node(node, source_length=1000)
        print(node.get_content(metadata_mode=MetadataMode.EMBED))
        print("\n")
    return retrieval_results

query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
check_retrieval_results(query)

Generated queries:
grocery stores nashville curated items unique local products
"I recently discovered a fantastic grocery store in Nashville that has an amazing selection of curated items and unique local products. The atmosphere is inviting, and the staff is incredibly knowledgeable about their offerings."


**Node ID:** 2eadad41-5c0b-4023-8f59-b8c38bfccfaa<br>**Similarity:** 0.0548742496770762<br>**Text:** Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners

Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Pa

**Node ID:** b8be6ef5-1fac-4bc8-9c4e-6c467039ff01<br>**Similarity:** 0.05246854594680682<br>**Text:** This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners

This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.




...

**Node ID:** bda4e1ea-4228-440d-8982-d15f82738c9a<br>**Similarity:** 0.02295788939624556<br>**Text:** (Does that mean I'm a believer in inorganic food, whatever that is? You decide...) 

I go to Whole Foods to get some items that are just not readily available in my regular grocery store - CostCo. Items like Arctic Char, Lake Trout, some cheeses, some breads, etc. I'm not sure where else I would go in town for more specialty foods like that, and Whole Foods seems as good of a place as anywhere else.

The overall selection is pretty good, if you buy all of your groceries from here. Prices are higher than CostCo, but you knew that.<br>

business_id: 0vCsslN-TYnDeOSNT4rIfg
review_stars: 4
biz_name: Whole Foods Market
biz_address: 14598 Clay Terrace Blvd
biz_city: Carmel
biz_state: IN
biz_categories: Food, Organic Stores, Specialty Food, Health Markets, Grocery

(Does that mean I'm a believer in inorganic food, whatever that is? You decide...) 

I go to Whole Foods to get some items that are just not readily available in my regular grocery store - CostCo. Items like Arctic Char, Lake Trout, some cheeses, some breads, etc. I'm not sure where else I would go in town for more specialty foods like that, and Whole Foods seems as good of a place as anywhere else.

The overall selection is pretty good, if you buy all of your groceries from here. Prices are higher than CostCo, but you knew that.




**Node ID:** 5ca5150d-405c-4397-b1ae-9cbe8b4a600d<br>**Similarity:** 0.0226010101010101<br>**Text:** I'm a fan of this teeny-tiny Kroger because I love being able to walk to the grocery store with my little reusable bags. Okay, this Kroger does not have the greatest selection, but they do have staples and some pretty nice high-end items, cheeses included. The tuna steaks I purchased are beautiful, and at a really good price.

The wine selection is awesome for such a small space!

I really wish they carried Athenos Greek yogurt, because they carry all kinds of Athenos products. I also really wish the cashiers were nicer.<br>

business_id: cogiJiOXN5fDu7jnkYEkfQ
review_stars: 4
biz_name: Kroger
biz_address: 6220 Guilford Ave
biz_city: Indianapolis
biz_state: IN
biz_categories: Grocery, Food

I'm a fan of this teeny-tiny Kroger because I love being able to walk to the grocery store with my little reusable bags. Okay, this Kroger does not have the greatest selection, but they do have staples and some pretty nice high-end items, cheeses included. The tuna steaks I purchased are beautiful, and at a really good price.

The wine selection is awesome for such a small space!

I really wish they carried Athenos Greek yogurt, because they carry all kinds of Athenos products. I also really wish the cashiers were nicer.




[NodeWithScore(node=TextNode(id_='2eadad41-5c0b-4023-8f59-b8c38bfccfaa', embedding=None, metadata={'review_id': 'VZMjK-Moy-fN08mR2m9JUg', 'user_id': 'U5hfpmnqW65_u10C1i06Dw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 4, 'funny': 1, 'cool': 0, 'date': '2017-06-23T12:16:51', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': Fals

# Response

In [43]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.core.response_synthesizers.type import ResponseMode
from llama_index.core.query_engine import CitationQueryEngine

from src.features.citation.custom_citation_query_engine import CUSTOM_CITATION_QA_TEMPLATE, CUSTOM_CITATION_REFINE_TEMPLATE

In [44]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k, use_fp16=True)
node_postprocessors.append(reranker)

query_engine = CitationQueryEngine.from_args(
    index=index,
    retriever=retriever,
    node_postprocessors=node_postprocessors,
    citation_qa_template=CUSTOM_CITATION_QA_TEMPLATE,
    citation_refine_template=CUSTOM_CITATION_REFINE_TEMPLATE
)

## Test Query Engine

In [45]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [46]:
query = "What are some gastropubs in Indianapolis that are known for their cocktails?"
# query = "What are some places to enjoy cold brew coffee"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
gastropubs cocktails indianapolis
"I visited a fantastic gastropub in Indianapolis called The Libertine Liquor Bar, which is famous for its creative cocktails and vibrant atmosphere. They have a great selection of drinks that really stand out."


**`Final Response:`** Based on your query about gastropubs in Indianapolis that are known for their cocktails, here are several recommendations ranked by relevance:

### 1. BODHI: Craft Bar + Thai Bistro
- **Address:** 922 Massachusetts Ave, Indianapolis, IN
- **Categories:** Gastropubs, Nightlife, Restaurants, Thai, Bars, Cocktail Bars
- **Summary:** BODHI is highly regarded for its amazing cocktails and delicious Thai cuisine. Reviewers have praised the extensive cocktail menu and the quality of the food, particularly the pad Thai and crab Rangoon. The atmosphere is vibrant, making it a great spot for both dining and enjoying drinks with friends [2][3][5]. If you're looking for a place that combines a unique culinary experience with excellent cocktails, BODHI is a top choice.

### 2. The Libertine Liquor Bar
- **Address:** 38 E Washington St, Indianapolis, IN
- **Categories:** Restaurants, Lounges, Bars, American (New), Nightlife
- **Summary:** The Libertine is celebrated as one of the best cocktail bars in Indianapolis, known for its knowledgeable bartenders and inventive cocktails. The food menu is extensive, offering both small plates and full entrees, with standout dishes like elote pappardelle. The atmosphere is classy and perfect for a night out or entertaining guests [1][9]. If you prioritize a sophisticated cocktail experience alongside a diverse menu, The Libertine is an excellent option.

### 3. Village Tap Room
- **Address:** 838 Broad Ripple Ave, Indianapolis, IN
- **Categories:** Gastropubs, Cocktail Bars, Beer Bar, Bars, Restaurants, Nightlife
- **Summary:** Village Tap Room has been noted for its selection of local craft beers and craft cocktails. While it may not be as widely recognized for its cocktail menu as BODHI or The Libertine, it offers a solid food menu and a relaxed atmosphere, making it a good choice for casual outings [4]. If you're looking for a laid-back environment with good drinks, this could be a suitable option.

### Comparison of Options:
- **BODHI** stands out for its combination of Thai cuisine and cocktails, making it ideal for those who want a unique dining experience alongside their drinks.
- **The Libertine** excels in cocktail craftsmanship and a more upscale dining experience, perfect for special occasions or impressing guests.
- **Village Tap Room** offers a more casual vibe with a focus on local beers and cocktails, making it suitable for a relaxed night out.

In summary, if you are looking for a vibrant atmosphere with a unique culinary twist, BODHI is your best bet. For a more upscale cocktail experience, The Libertine is unmatched. If you prefer a casual setting with a good selection of drinks, consider Village Tap Room.

Sources:
- [1]: "Libertine is an Indianapolis gem and one of the best places for a cocktail in the city."
- [2]: "Came here and got cocktails and dessert. Their cocktails are amazing! Highly recommend."
- [3]: "I love pad Thai and their pad Thai is one of the best I've had in Indy."
- [4]: "Much improved over Chumley's. Nice selection of local craft beer. Craft cocktails. Food was very good."
- [5]: "Happy to see this one on the block!"

---

**`Source Node 1/10`**

**Node ID:** 457bf280-c39c-43e9-9369-f5b9f39caccc<br>**Similarity:** 2.08203125<br>**Text:** Source 1:
Libertine is an Indianapolis gem and one of the best places for a cocktail in the city.  The other reviewers are right- sit at the bar!  The bartenders are knowledgeable and skilled; they offer great drink suggestions to make your experience feel more personal.  Libertine serves up 5-star cocktails and under chef Neal Brown, the food is just as impressive.  The menu is extensive for a cocktail lounge, offering small plates if you're snacking or full entrees for dinner.  Last night I had the elote pappardelle: buttery pappardelle pasta with sweet corn, tomatoes, & cotija cheese.   It was unlike anything I've had before and so tasty! If you're looking for inventive cuisine and cocktails to sip and savor in a classy gathering place- head to Libertine. Bring your out-of-town guests.<br>**Metadata:** {'review_id': 'uYVhESHLFdAkMlhjYdTZYA', 'user_id': 'XCydwvWxRidDyHKetxx8xw', 'business_id': 'v5kwdvmg-J8g-ll0nkc3Sw', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2014-12-20T23:13:56', 'biz_name': 'The Libertine Liquor Bar', 'biz_address': '38 E Washington St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46204', 'biz_latitude': 39.7672488945, 'biz_longitude': -86.1567753553, 'biz_stars': 4.0, 'biz_review_count': 208, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': True, \'upscale\': False, \'casual\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': False, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'False\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Lounges, Bars, American (New), Nightlife'}<br>

...

**Node ID:** bb46f528-a1a9-4381-8a2a-37dbeaa0a078<br>**Similarity:** 0.31787109375<br>**Text:** Source 9:
I will say that I am always on the search to find places that make great craft cocktails and The Libertine fits the bill.  Excellent craft cocktails, very interesting menu choices and a cool atmosphere make The Libertine an Indianapolis staple.

The mixologists here have comprehensive craft cocktail knowledge and produce some wonderful, tasty cocktails. If you want something made with some bottled sour mix or drowning in ice, you may need to go somewhere else.

The menu has some wonderful items as well.  How about trying out their bacon flight with different kinds of bacon and dipping sauces? Yes please!<br>**Metadata:** {'review_id': 'w1umwRm7KJEw3iGZ4NIYTA', 'user_id': 'nA-SGbVpx7CAYfU3UHSgJQ', 'business_id': 'v5kwdvmg-J8g-ll0nkc3Sw', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 2, 'date': '2014-04-24T00:29:42', 'biz_name': 'The Libertine Liquor Bar', 'biz_address': '38 E Washington St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46204', 'biz_latitude': 39.7672488945, 'biz_longitude': -86.1567753553, 'biz_stars': 4.0, 'biz_review_count': 208, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': True, \'upscale\': False, \'casual\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': False, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'False\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Lounges, Bars, American (New), Nightlife'}<br>

---

**`Source Node 10/10`**

**Node ID:** e4c8aeed-a8f5-47dd-9c20-57d219488098<br>**Similarity:** 0.2000732421875<br>**Text:** Source 10:
There are a few great cocktail bars in Indianapolis and this is one of them. The bar decor is comfortable with lots of wood and an interesting back bar.<br>**Metadata:** {'review_id': 'EyfS0mZGAP6yIGe-jVHdfQ', 'user_id': 'e7qswoJFIolWdf1qX_P_oA', 'business_id': 'v5kwdvmg-J8g-ll0nkc3Sw', 'review_stars': 4, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2013-12-16T12:14:22', 'biz_name': 'The Libertine Liquor Bar', 'biz_address': '38 E Washington St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46204', 'biz_latitude': 39.7672488945, 'biz_longitude': -86.1567753553, 'biz_stars': 4.0, 'biz_review_count': 208, 'biz_is_open': 0, 'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': True, \'upscale\': False, \'casual\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': False, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'False\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Lounges, Bars, American (New), Nightlife'}<br>

{'457bf280-c39c-43e9-9369-f5b9f39caccc': {'review_id': 'uYVhESHLFdAkMlhjYdTZYA',
  'user_id': 'XCydwvWxRidDyHKetxx8xw',
  'business_id': 'v5kwdvmg-J8g-ll0nkc3Sw',
  'review_stars': 5,
  'useful': 1,
  'funny': 0,
  'cool': 1,
  'date': '2014-12-20T23:13:56',
  'biz_name': 'The Libertine Liquor Bar',
  'biz_address': '38 E Washington St',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46204',
  'biz_latitude': 39.7672488945,
  'biz_longitude': -86.1567753553,
  'biz_stars': 4.0,
  'biz_review_count': 208,
  'biz_is_open': 0,
  'biz_attributes': '{\'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': True, \'upscale\': False, \'casual\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\'

## Response Evaluation

In [47]:
from src.run.eval import ResponseEvaluator

In [48]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [49]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-09-08 08:35:02.151 | INFO     | src.run.eval.response:generate_synthetic_dataset:40 - Sampling 30 documents for response evaluation...
2024-09-08 08:35:02.197 | INFO     | src.run.eval.response:generate_synthetic_dataset:96 - Loading existing synthetic response eval dataset at data/034_rerun_400_restaurants/response_synthetic_eval_dataset.json...


In [50]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

Generated queries:
Caribbean restaurants Indianapolis flavorful dishes
"I recently dined at a fantastic Caribbean restaurant in Indianapolis that had an amazing selection of flavorful dishes. The jerk chicken was spicy and delicious, and the plantains were perfectly cooked. I highly recommend it!"


Batch processing of predictions:   0%|                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

Generated queries:
seafood restaurants santa barbara fresh dishes reasonable prices
"I visited The Fish Market in Santa Barbara and was impressed by their fresh seafood options at great prices. The atmosphere was casual, and the service was friendly."


Batch processing of predictions:   0%|                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

...

Batch processing of predictions:   0%|                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

Generated queries:
steakhouses flavorful steaks New Orleans
"I visited Dickie Brennan's Steakhouse in New Orleans, and their steaks were incredibly flavorful. The ambiance was great, and the service was top-notch!"


Batch processing of predictions:   0%|                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

Generated queries:
good beer selection pubs quick service


Batch processing of predictions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.15s/it]


In [51]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [02:40<00:00,  5.17s/it]


In [52]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.483871
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_relevancy_score,0.665323


In [53]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are some Caribbean restaurants in Indiana...,Based on your query about Caribbean restaurant...,1.0,5.0,1.0,0.625,[Source 1:\nThis review is for their establish...
1,- What are some seafood restaurants in Santa B...,Based on your query about seafood restaurants ...,1.0,5.0,1.0,0.875,[Source 1:\nThis is the must visit if you are ...
2,- What are some hotels in Nashville that offer...,Based on your query about hotels in Nashville ...,1.0,4.0,1.0,0.750,[Source 1:\nCome and see the gardens a couple ...
3,- What are some highly-rated seafood restauran...,Based on your query about highly-rated seafood...,1.0,5.0,1.0,0.625,[Source 1:\nThe drinks are a step above dive b...
4,What are some highly-rated fast food restauran...,Based on your query about highly-rated fast fo...,1.0,4.0,1.0,0.625,[Source 1:\nIt was an interesting experience. ...
5,- What are some Indian restaurants known for t...,Based on your query about Indian restaurants k...,1.0,4.0,1.0,0.625,[Source 1:\nMy favorite Indian restaurant in T...
6,What are some pizza restaurants in Tampa known...,Based on your query about pizza restaurants in...,1.0,5.0,1.0,0.750,[Source 1:\nThis is the number 1 pizza restaur...
7,- What are some seafood restaurants in Santa B...,Based on your query about seafood restaurants ...,1.0,5.0,1.0,0.875,[Source 1:\nOMG\nPassing through Santa Barbara...
8,What are some highly-rated bars in Nashville t...,Based on your query about highly-rated bars in...,1.0,4.0,1.0,0.625,[Source 1:\navailable for your convenience. ...
9,- What are some popular coffee roasteries in S...,Based on your query about popular coffee roast...,1.0,5.0,1.0,0.500,[Source 1:\nThis is a fantastic spot for a str...


#### Error Analysis

In [54]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [55]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score', 'context_relevancy_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
16,- What are some Indian or Pakistani restaurant...,Based on your query about Indian or Pakistani ...,1.0,2.0,1.0,0.625,[Source 1:\nAmber Indian is easily the best ch...
12,- What are some pubs in Exton that are known f...,Based on your query about pubs in Exton known ...,1.0,3.0,1.0,0.250,"[Source 1:\nGreat selection of beers, friendly..."
28,What are some venues or event spaces that are ...,Based on your query about venues or event spac...,1.0,3.0,1.0,0.625,[Source 1:\nThis place is huge. Everything is ...
21,What are some hotels in Nashville that have a ...,Based on your query about hotels in Nashville ...,1.0,4.0,1.0,0.500,[Source 1:\nThat is NOT good customer service....
4,What are some highly-rated fast food restauran...,Based on your query about highly-rated fast fo...,1.0,4.0,1.0,0.625,[Source 1:\nIt was an interesting experience. ...
5,- What are some Indian restaurants known for t...,Based on your query about Indian restaurants k...,1.0,4.0,1.0,0.625,[Source 1:\nMy favorite Indian restaurant in T...
8,What are some highly-rated bars in Nashville t...,Based on your query about highly-rated bars in...,1.0,4.0,1.0,0.625,[Source 1:\navailable for your convenience. ...
26,- What are some great places to grab a quick m...,Based on your query about great places to grab...,1.0,4.0,1.0,0.625,[Source 1:\nI would also recommend this for an...
27,- What are some highly-rated restaurants in Vi...,Based on your query for highly-rated restauran...,1.0,4.0,1.0,0.625,"[Source 1:\nHeidi, Judy and Dustin are outstan..."
2,- What are some hotels in Nashville that offer...,Based on your query about hotels in Nashville ...,1.0,4.0,1.0,0.750,[Source 1:\nCome and see the gardens a couple ...


In [56]:
CONTEXT_RELEVANCE_SCORE_THRESHOLD = 0.5

response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['context_relevancy_score'].le(CONTEXT_RELEVANCE_SCORE_THRESHOLD)]
    .sort_values(['context_relevancy_score', 'relevancy_score', 'correctness_score', 'faithfulness_score'])
)

with pd.option_context('display.max_colwidth', 400):
    display(response_synthetic_eval_irrelevance_df)

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
12,- What are some pubs in Exton that are known for their quality cocktails?,"Based on your query about pubs in Exton known for their quality cocktails, here are several recommendations ranked by relevance:\n\n### 1. World of Beer\n- **Address:** 102 Main St, Ste 100, Exton, PA\n- **Categories:** Nightlife, Pubs, American (Traditional), Restaurants, Bars\n- **Summary:** World of Beer is a popular spot in Exton, known for its extensive beer selection and a decent cocktai...",1.0,3.0,1.0,0.25,"[Source 1:\nGreat selection of beers, friendly staff. Overall I highly recommend WOB in Exton\n, Source 2:\nLet me tell you about this place. I've been here a couple of times, the last one being with my brother and sister after getting our picture with the Easter Bunny in the Exton Mall. Now of course they have an extensive and creative beer list; it's called World of Beer™ for crying out lou..."
22,What are some hotels in Nashville that offer better customer service than Gaylord Opryland Resort & Convention Center?,"Based on your query about hotels in Nashville that offer better customer service than the Gaylord Opryland Resort & Convention Center, here are several recommendations ranked by relevance:\n\n### 1. The Hermitage Hotel\n- **Address:** 231 6th Ave N, Nashville, TN\n- **Categories:** Luxury Hotel, Historic Hotel, American (New), Restaurants, Bars\n- **Summary:** The Hermitage Hotel is renowned f...",1.0,5.0,1.0,0.25,"[Source 1:\nThe Gaylord is magnificent! They are located in Kissimmee, FL, Grapevine, TX, Aurora, CO, National Harbor, MD, and Nashville, TN. I visited the Nashville location. This was my first time ever visiting a Gaylord Opryland Resort. \n\nTennessee's Gaylord Opryland Resort & Convention Center is the largest non-gaming in-hotel facility in the world. It contains 2,711 guest rooms and 171 ..."
21,What are some hotels in Nashville that have a reputation for good customer service?,"Based on your query about hotels in Nashville that have a reputation for good customer service, here are several recommendations ranked by relevance:\n\n### 1. Gaylord Opryland Resort & Convention Center\n- **Address:** 2800 Opryland Dr, Nashville, TN\n- **Categories:** Venues & Event Spaces, Performing Arts, Arts & Entertainment, Hotels & Travel, Food, Convenience Stores, American (New), Beau...",1.0,4.0,1.0,0.50,[Source 1:\nThat is NOT good customer service. That is NOT good business. \n\nTreating customers like billing casualties is disgraceful. \n\nNo - I do not want to press 1 to speak to a customer service representative who isn't on the property and can't do anything to fix my issue. \nNo - I do not want to wait 5-7 business days and try to figure out how I'm going to pay my mortgage sans the $$$...
9,- What are some popular coffee roasteries in Santa Barbara that offer a variety of drinks?,"Based on your query about popular coffee roasteries in Santa Barbara that offer a variety of drinks, here are several recommendations ranked by relevance:\n\n### 1. Dune Coffee Roasters - Anacapa\n- **Address:** 528 Anacapa St, Santa Barbara, CA\n- **Categories:** Coffee & Tea, Coffee Roasteries, Food\n- **Summary:** Dune Coffee Roasters is highly regarded for its exceptional coffee and divers...",1.0,5.0,1.0,0.50,"[Source 1:\nThis is a fantastic spot for a strong cup of Joe! The French Press is a bustling coffee shop in Santa Barbara that attracts locals and tourists alike. There are plenty of beverage options, but more uniquely plenty of tasty snacks to choose from... Like several different types of toast and spreads!\n, Source 2:\nThe French Press has the best espresso drinks in Santa Barbara. I hear..."
11,- What are some sports bars in Indianapolis that offer a good variety of American cuisine?,"Based on your query about sports bars in Indianapolis that offer a good variety of American cuisine, here are several recommendations ranked by re

In [57]:
for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
- What are some pubs in Exton that are known for their quality cocktails?

Context:
Source 1:
Great selection of beers, friendly staff.  Overall I highly recommend WOB in Exton


Source 2:
Let me tell you about this place. I've been here a couple of times, the last one being with my brother and sister after getting our picture with the Easter Bunny in the Exton Mall. Now of course they have an extensive and creative beer list; it's called World of Beer™ for crying out loud. What's tricky is the food. A bar/restaurant known for their drinks can be a hit or miss in regards to the culinary portion of the meal. Fortunately, this place is a hit. Their burgers are cooked just the way you like, with that hint of love mama put into her home cooking when she wasn't drunk and gambling your college savings away with Dan. And that Bavarian pretzel? First time I saw that delicious monstrosity, I muttered, "Oh yes daddy."


Source 3:
I started frequen

In [58]:
# query = "- What are some pizza places known for their quality and flavor in White House, TN?"
query = "What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a sports bar in Indianapolis that had an amazing atmosphere and delicious food. The wings were fantastic, and they offered a great selection of craft beers. Definitely a step up from Twin Peaks!"


**`Final Response:`** Based on your query about sports bars in Indianapolis that offer a better dining experience than Twin Peaks, here are several recommendations ranked by relevance:

### 1. **The Eagle**
- **Address:** 310 Massachusetts Ave, Indianapolis, IN
- **Categories:** American (New), Sports Bars, Restaurants
- **Summary:** The Eagle is known for its fried chicken and Southern-inspired menu, offering a more upscale dining experience compared to typical sports bars. The atmosphere is casual yet refined, making it suitable for families and groups. The restaurant also features a good selection of craft beers and cocktails, enhancing the overall dining experience.

### 2. **Buffalo Wild Wings**
- **Address:** 1001 E 86th St, Indianapolis, IN
- **Categories:** Sports Bars, American (Traditional), Restaurants
- **Summary:** Buffalo Wild Wings is a well-known chain that provides a lively atmosphere with numerous TVs for sports viewing. The menu is extensive, featuring wings, burgers, and a variety of appetizers. While it may not be as upscale as The Eagle, it offers a fun environment with a focus on sports and casual dining.

### 3. **The Slippery Noodle Inn**
- **Address:** 372 S Meridian St, Indianapolis, IN
- **Categories:** Live Music, Sports Bars, American (Traditional), Restaurants
- **Summary:** The Slippery Noodle Inn is Indiana's oldest bar and offers a unique blend of live music and sports viewing. The menu features classic American fare, and the historic ambiance adds to the dining experience. This venue is perfect for those looking to enjoy good food while also experiencing local music.

### 4. **The Rathskeller**
- **Address:** 401 E Michigan St, Indianapolis, IN
- **Categories:** German, Beer Garden, Sports Bars, Restaurants
- **Summary:** The Rathskeller offers a unique German dining experience with a large beer garden and a variety of German beers. The atmosphere is vibrant, and the menu includes traditional German dishes alongside American favorites. This venue is ideal for those looking for a different cultural experience while watching sports.

### Comparison of Options:
- **The Eagle** stands out for its upscale dining experience and quality food, making it the top recommendation for those seeking a better dining experience than Twin Peaks.
- **Buffalo Wild Wings** is more casual and focused on sports, making it a great choice for a lively atmosphere but may not match the dining quality of The Eagle.
- **The Slippery Noodle Inn** offers a unique combination of live music and sports, appealing to those who enjoy entertainment alongside their meal.
- **The Rathskeller** provides a cultural twist with its German menu and beer garden, making it a fun option for those looking to try something different.

In summary, if your priority is a better dining experience, **The Eagle** is the best choice. For a lively sports atmosphere, **Buffalo Wild Wings** is a solid option, while **The Slippery Noodle Inn** and **The Rathskeller** offer unique experiences that combine food with entertainment.

Sources:
- No specific sources were cited as the query did not provide relevant reviews for the recommended places.

---

**`Source Node 1/11`**

**Node ID:** ad69b6dc-67a1-4cc4-8c83-33d52d88675a<br>**Similarity:** 0.86279296875<br>**Text:** Source 1:
Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.<br>**Metadata:** {'review_id': 'KsmnyFmghtOx_IbOG9RINg', 'user_id': 'jB-7q5FxZKho-wlRAzS2lA', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2017-10-25T03:00:01', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

...

**Node ID:** 307893fd-5d94-4c7d-870d-d8fade96a11f<br>**Similarity:** -0.381591796875<br>**Text:** Source 10:
It's my cuz per   Bachelor party.   The  Waitress at twin peaks (Tyler Walker) is the best and she is a very good waitress. Plus Tyler works hard at all  her tables. Just want the thanks Twin  peaks and Tyler Walker for a very good time.<br>**Metadata:** {'review_id': '8j70_2c-O_L2lWa0nz7KDQ', 'user_id': 'nmqDutrkq9tw_ZrOlLr0FQ', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-09-09T02:37:35', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

---

**`Source Node 11/11`**

**Node ID:** b4ad17dc-55c7-4b2f-b99e-9032eba3d212<br>**Similarity:** -0.445556640625<br>**Text:** Source 11:
Been wanting to come to twin peaks for a while. The food was great, and the service, FROM VICTORIA was amazing.<br>**Metadata:** {'review_id': 'uOzC-IXnlc5hJ2h-0jNRKQ', 'user_id': 'ZphdnOBaO5Qd3STxXjspVw', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2022-01-18T20:36:24', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

{'ad69b6dc-67a1-4cc4-8c83-33d52d88675a': {'review_id': 'KsmnyFmghtOx_IbOG9RINg',
  'user_id': 'jB-7q5FxZKho-wlRAzS2lA',
  'business_id': 'seKihQKpGGnCeLuELRQPSQ',
  'review_stars': 5,
  'useful': 1,
  'funny': 0,
  'cool': 0,
  'date': '2017-10-25T03:00:01',
  'biz_name': 'Twin Peaks',
  'biz_address': '6880 E 82nd St',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46250',
  'biz_latitude': 39.9062952,
  'biz_longitude': -86.0474634,
  'biz_stars': 3.5,
  'biz_review_count': 257,
  'biz_is_open': 1,
  'biz_attributes': '{\'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BYOBCorkage\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParkin

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [59]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-09-08 08:42:41.455 | INFO     | src.run.eval.response:generate_curated_dataset:125 - Persisting curated response eval dataset at data/034_rerun_400_restaurants/response_curated_eval_dataset.json...


In [60]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee. The atmosphere was perfect for relaxing, and they offered a fantastic selection of cold brew options that were incredibly refreshing."


Batch processing of predictions:   0%|                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

Generated queries:
gastropubs cocktails indianapolis
"I recently visited a gastropub in Indianapolis that had an amazing cocktail menu. The mixologist crafted unique drinks that paired perfectly with the food. Highly recommend checking out the atmosphere and the creative cocktails!"


Batch processing of predictions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.64s/it]


In [61]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.87s/it]


In [62]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.0000
mean_relevancy_score,1.0000
mean_faithfulness_score,1.0000
mean_context_relevancy_score,0.8125


In [63]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are some places to enjoy cold brew coffee?,Based on your query about places to enjoy cold...,1.0,4.0,1.0,0.875,[Source 1:\nRed Hook is my go-to coffee shop i...
1,What are some gastropubs in Indianapolis that ...,Based on your query about gastropubs in Indian...,1.0,4.0,1.0,0.750,[Source 1:\nLibertine is an Indianapolis gem a...


#### Answers for target questions

In [64]:
from IPython.display import Markdown

In [65]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [66]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are some places to enjoy cold brew coffee?


Generated queries:
cold brew coffee places
"I recently visited a café that specializes in cold brew coffee, and it was amazing! The atmosphere was cozy, and they had a wide selection of cold brew options that were perfect for a hot day."


**`Final Response:`** Based on your query about places to enjoy cold brew coffee, here are several recommendations ranked by relevance:

### 1. Red Hook Coffee & Tea
- **Address:** 765 S 4th St, Philadelphia, PA
- **Categories:** Restaurants, Breakfast & Brunch, Vegan, Bagels, Food, Coffee & Tea
- **Summary:** Red Hook Coffee & Tea is highly regarded for its intense and delicious cold brew. Reviewers consistently praise the quality of their coffee, including seasonal offerings like the maple rosemary latte, which is noted for its perfect balance of flavors. The café is a popular spot in the neighborhood, making it a great choice for both coffee enthusiasts and casual visitors [1].

### 2. Dune Coffee Roasters - Anacapa
- **Address:** 528 Anacapa St, Santa Barbara, CA
- **Categories:** Coffee & Tea, Coffee Roasteries, Food
- **Summary:** Dune Coffee Roasters is celebrated for its exceptional cold brew and iced mochas. The café features a spacious outdoor patio, allowing patrons to enjoy their drinks in a pleasant setting. Reviewers highlight the smoothness of the cold brew, which doesn't taste watered down, even as the ice melts. This location is ideal for those looking for a relaxing coffee experience [4][5][9].

### 3. Wawa
- **Address:** 3744 Spruce St, Philadelphia, PA
- **Categories:** Delis, Gas Stations, Food, Convenience Stores, Restaurants, Automotive, Coffee & Tea, Sandwiches
- **Summary:** While Wawa is primarily known as a convenience store, it offers a surprisingly good cold brew coffee. It’s a convenient stop for those on the go, especially during the summer months when cold beverages are in high demand. However, it may not provide the same ambiance as dedicated coffee shops [2].

### Comparison of Options:
- **Red Hook Coffee & Tea** stands out for its strong community presence and high-quality coffee, making it perfect for those who appreciate a cozy café atmosphere and a variety of drink options.
- **Dune Coffee Roasters** offers a more laid-back vibe with outdoor seating, ideal for enjoying coffee in a relaxed environment. Its focus on quality and unique offerings makes it a great choice for coffee aficionados.
- **Wawa**, while convenient and offering decent cold brew, lacks the charm and specialized focus of the other two options. It is best suited for quick stops rather than a dedicated coffee experience.

In summary, if you're looking for a dedicated coffee experience with a strong emphasis on quality, **Red Hook Coffee & Tea** and **Dune Coffee Roasters** are your best bets. For convenience, **Wawa** is a solid option, especially if you're in a hurry.

Sources:
- [1]: "Red Hook is my go-to coffee shop in the neighborhood. Their cold brew is intense and delicious..."
- [4]: "Heavily enjoy their cold brew and iced mochas."
- [5]: "I really liked that the cold brew didn't taste watered down even after the ice melted."
- [2]: "Cold Brew Coffee Quest #6..."



### Question: What are some gastropubs in Indianapolis that are known for their cocktails?


Generated queries:
gastropubs cocktails indianapolis
"I visited a fantastic gastropub in Indianapolis that had an amazing cocktail menu. The mixologist crafted unique drinks that perfectly complemented the delicious food. Definitely a place to check out for cocktail lovers!"


**`Final Response:`** Based on your query about gastropubs in Indianapolis that are known for their cocktails, here are several recommendations ranked by relevance:

### 1. BODHI: Craft Bar + Thai Bistro
- **Address:** 922 Massachusetts Ave, Indianapolis, IN
- **Categories:** Gastropubs, Nightlife, Restaurants, Thai, Bars, Cocktail Bars
- **Summary:** BODHI is highly regarded for its amazing cocktails and delicious Thai cuisine. Reviewers have praised the extensive cocktail menu and the quality of the food, particularly the pad Thai and crab Rangoon. The atmosphere is vibrant, making it a great spot for both dining and enjoying drinks. The combination of Thai food and craft cocktails makes it a unique experience in the downtown area [2][5].

### 2. The Libertine Liquor Bar
- **Address:** 38 E Washington St, Indianapolis, IN
- **Categories:** Restaurants, Lounges, Bars, American (New), Nightlife
- **Summary:** The Libertine is celebrated as one of the best cocktail bars in Indianapolis, known for its 5-star cocktails and inventive cuisine. The bartenders are skilled and offer personalized drink suggestions, enhancing the overall experience. The menu includes a variety of small plates and full entrees, making it suitable for both casual snacking and full meals. The atmosphere is classy, making it a perfect spot for a night out or entertaining guests [1][9].

### 3. Village Tap Room
- **Address:** 838 Broad Ripple Ave, Indianapolis, IN
- **Categories:** Gastropubs, Cocktail Bars, Beer Bar, Bars, Restaurants, Nightlife
- **Summary:** Village Tap Room has been noted for its selection of local craft beers and craft cocktails. While it may not be as cocktail-focused as BODHI or The Libertine, it offers a solid food menu and a relaxed atmosphere. This makes it a good option for those looking to enjoy a casual night out with friends while still having access to quality drinks [4].

### Comparison of Options:
- **BODHI** stands out for its unique blend of Thai cuisine and craft cocktails, making it ideal for those who want a diverse dining experience alongside their drinks. It is particularly recommended for those who enjoy Asian flavors and a lively atmosphere.
  
- **The Libertine** is perfect for cocktail enthusiasts who appreciate a more upscale environment and a focus on innovative drink options. It is a great choice for special occasions or when you want to impress guests with a sophisticated dining experience.

- **Village Tap Room** is more casual and beer-focused, making it suitable for a laid-back outing. It may not have the same level of cocktail creativity as the other two, but it offers a good selection of drinks and a friendly atmosphere.

In summary, if you prioritize cocktails and a unique dining experience, BODHI is your best bet. For a more upscale cocktail experience, The Libertine is the way to go. If you're looking for a casual spot with good drinks, consider Village Tap Room.

### Sources:
- [1]: "Libertine is an Indianapolis gem and one of the best places for a cocktail in the city."
- [2]: "Came here and got cocktails and dessert. Their cocktails are amazing! Highly recommend."
- [4]: "Much improved over Chumley's. Nice selection of local craft beer. Craft cocktails. Food was very good."
- [5]: "Happy to see this one on the block!" 
- [9]: "Excellent craft cocktails, very interesting menu choices and a cool atmosphere make The Libertine an Indianapolis staple."

### Error Analysis

In [67]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [68]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are some places to enjoy cold brew coffee?,Based on your query about places to enjoy cold...,1.0,4.0,1.0,0.875,[Source 1:\nRed Hook is my go-to coffee shop i...
1,What are some gastropubs in Indianapolis that ...,Based on your query about gastropubs in Indian...,1.0,4.0,1.0,0.750,[Source 1:\nLibertine is an Indianapolis gem a...


In [69]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [70]:
from src.run.utils import parse_collect_log, flatten_dict

In [71]:
collect_info = parse_collect_log(collect_fp)

In [72]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

2024-09-08 08:43:39.965 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-09-08 08:43:40.056 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-09-08 08:43:40.825 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-09-08 08:43:41.356 | INFO     | __main

# Clean up

In [73]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Chat Bot Quick Test

In [74]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from src.svc.availability.availability_check import ReservationService

In [75]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="reco_review",
        description=(
            "useful for when you want to find places to visit"
            " based on end-user reviews"
        ),
    ),
)

opening_hours_db = data.set_index('business_id')['biz_hours'].dropna().to_dict()
biz_name_id_mapper = data.set_index('biz_name')['business_id'].dropna().to_dict()
rez_tool = ReservationService(opening_hours_db, biz_name_id_mapper)

tools = [query_engine_tool] + rez_tool.to_tool_list()

In [76]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [77]:
response = agent.chat("hi, i am quy")
print(str(response))

response = agent.chat(
    "Where to find some good cold brew coffee?"
)
print(str(response))

response = agent.chat(
    "where is it?"
)
print(str(response))

response = agent.chat(
    "what is the best time to visit?"
)
print(str(response))

Added user message to memory: hi, i am quy
Hello Quy! How can I assist you today?
Added user message to memory: Where to find some good cold brew coffee?
=== Calling Function ===
Calling function: reco_review with args: {"input":"cold brew coffee"}
Generated queries:
cold brew coffee
"I recently tried a new cafe that specializes in cold brew coffee, and I was blown away by their smooth and rich flavors. Definitely a must-visit for any coffee lover!"
Got output: Based on your query about cold brew coffee, here are several recommendations ranked by relevance:

### 1. Dune Coffee Roasters - Anacapa
- **Address:** 528 Anacapa St, Santa Barbara, CA
- **Categories:** Coffee & Tea, Coffee Roasteries, Food
- **Summary:** Dune Coffee Roasters is highly regarded for its cold brew coffee, with many reviews praising its smoothness and flavor. One reviewer noted that the cold brew is a great pick-me-up at any time of day, highlighting its deliciousness [2]. The café also offers a variety of vegan m

In [78]:
response = agent.chat(
    "Can I book for 5 pax on August 15 at 8 AM?"
)
print(str(response))

Added user message to memory: Can I book for 5 pax on August 15 at 8 AM?
=== Calling Function ===
Calling function: check_availability with args: {"business_id":"Dune Coffee Roasters","date_time":"2023-08-15T08:00:00","pax":5}
Got output: False

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name":"Dune Coffee Roasters"}
Got output: None

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name":"Dune Coffee Roasters - Anacapa"}
Got output: -kY_HDP7IMvGl-kBIZVU4A

=== Calling Function ===
Calling function: check_availability with args: {"business_id":"-kY_HDP7IMvGl-kBIZVU4A","date_time":"2023-08-15T08:00:00","pax":5}
Got output: False

Unfortunately, it seems that there is no availability for a booking of 5 people at Dune Coffee Roasters on August 15 at 8 AM. 

If you would like, I can help you find another time or suggest a different coffee shop. Let me know!


In [79]:
agent.reset()

# Chatbot UI

# Archive